Tristan MANIER - Léa GRASSIEN - Mathieu RAMEL - INFO 5 - 9/01/2024


# 1. Présentation des données

Pourquoi avons nous choisis ces données ?
TODO

source, conditions de recueil, nombre de fichiers, nombres de variables et d'individus, expliquer l'activité derrière les données.
TODO

Donnez la signification desindividus et des variables
TODO

## Questions

### Analyse
Nous allons chercher à répondre à ces questions :
- Quelles sont les sources de production d'énergie les plus corrélées ?
- Quelles est la proportion d'énergie renouvelable produite ?
- Quelles sont les tendances de méthode de production d'énergie au cours de l'année ?
- Quelles sont les périodes de l'année où la production d'énergie est la plus importante/faible ?
- Quelles sont les moments de la journée où la production d'énergie est la plus importante/faible ?
- La météo influe-t-elle sur la consomation d'énergie ?
- Quelles sont les sources de production d'énergie les plus corrélées avec la météo ?

### Prédictions
- Quelle est la tendance de production d'énergie pour les jours/mois/années à venir ?
- 

# 2. Préparation des données

Ce notebook présente les commandes utiles pour les principales tâches de préparation des données. Les données `notes.data` qui servent d'illustration sont des notes obtenues par des étudiants dans des matières.

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 100)     # Set the maximum number of rows displayed to 100 rows

## 2.1 Premier dataset sur la production d'énergie

In [ ]:
df = pd.read_csv("eCO2mix_RTE_En-cours-TR.csv", sep="\t", encoding='latin-1', index_col=False)

df.head()

In [ ]:
print(f"{df.shape[0]} rows and {df.shape[1]} columns")

# Combine dtypre, count and nnunique
pd.concat([df.dtypes, df.count(), df.nunique()], keys=["Types", "Count", "NUnique"], axis=1)

### Statistiques descriptives des valeurs non-définies

In [ ]:
temp = ({
    'column':[],
    'nb_lines' :[],
    'nb_ND':[]
})

nd = pd.DataFrame(temp)

for column in df.columns : 
    new_row = pd.DataFrame({'column': column, 'nb_lines': df[column].count(), 'nb_ND': (df[column] == 'ND').sum()}, index=[0])
    nd = pd.concat([new_row, nd.loc[:]]).reset_index(drop=True)
nd = nd.sort_values(by='nb_ND', ascending=False)

nd["%ND"] = nd["nb_ND"] / nd["nb_lines"] * 100

nd

## 2.2 Deuxième dataset sur les données météo

In [ ]:
df_weather = pd.read_csv("donnees-synop-essentielles-omm.csv", sep=";", index_col=False, encoding='utf-8')

# df_weather.head()
print(f"{df_weather.shape[0]} rows and {df_weather.shape[1]} columns")

df_weather.head()

Selection des données d'une seule région. (Pourais être remplacer par une moyenne des régions) 

In [ ]:
code_region = int('07149')  # Ile-de-France
df_weather = df_weather[df_weather["numer_sta"] == code_region]

# df_weather.head()
print(f"{df_weather.shape[0]} rows and {df_weather.shape[1]} columns")

Formattage des données météo

In [ ]:
#multiplier les colonnes par 3 pour avoir 1 lignes par heures
weather_rename_map = {
    'date': 'date',
    'pmer': 'pression_mer',
    'ff': 'vitesse_vent',
    't': 'température',
    'u': 'humidité',
    'pres': 'pression',
    'niv_bar': 'niveau_barometrique',
    'tn12': 't_min_12h',
    'tn24': 't_min_24h',
    'tx12': 't_max_12h',
    'tx24': 't_max_24h',
    'tminsol': 't_min_sol_12h',
    'rr1': 'précipitation_1h',
    'rr3': 'précipitation_3h',
    'rr6': 'précipitation_6h',
    'rr12': 'précipitation_12h',
    'rr24': 'précipitation_24h',
    'ssfrai': 'hauteur_neige'
}

df_weather = df_weather.rename(columns=weather_rename_map)

for col_name in df_weather.columns.values.tolist():
    if col_name not in weather_rename_map.values():
        df_weather = df_weather.drop(col_name, axis=1)

df_weather.head()

Suppresion des lignes avec des valeurs manquantes.

In [ ]:
df = df.dropna(subset=['Date', "Heures"])

## 2.3 Jointure des deux datasets

Formatage des dates et heures des deux datasets.

In [ ]:
# Concat date and hours
df['date'] = df['Date'].astype(str) + df['Heures'].astype(str)

# Convert to datetime
df['date'] = df['date'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d%H:%M'))

df['date'].head()

In [ ]:
df_weather["date"] = df_weather['date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d%H%M%S'))
df_weather["date"]

Fusion des deux datasets

In [ ]:
df_full = pd.merge(df, df_weather, how='inner', on = 'date' )

# full_df.head()
print(f"df : {df.shape[0]} rows and {df.shape[1]} columns")
print(f"df_weather : {df_weather.shape[0]} rows and {df_weather.shape[1]} columns")
print(f"df_full : {df_full.shape[0]} rows and {df_full.shape[1]} columns")

# 3. Analyse des données

Selection d'une periode de temps

In [ ]:
from matplotlib import pyplot as plt

#select rows between date1 and date 2
date1 = pd.to_datetime('2022-06-01')
date2 = pd.to_datetime('2023-07-02')

df_selection = df_full[(df_full['date'] >= date1) & (df_full['date'] < date2)]

Visualisation des données de consomations

In [ ]:
figure = plt.figure(figsize=(20,10))
plt.plot(df_selection["date"], df_selection["Consommation"], label=column)
plt.legend( loc='upper right', numpoints = 1 )
plt.xlabel('Date')
plt.ylabel('Production (MW)')
plt.show()

Visualisation des données de production

In [ ]:

print()

to_plot = ["Charbon", "Eolien", "Solaire", "Nucléaire", "Hydraulique", "Bioénergies", "Pompage"]

figure = plt.figure(figsize=(20,10))
for column in to_plot :
    plt.plot(df_selection["date"], df_selection[column], label=column)
    plt.legend( loc='upper right', numpoints = 1 )

plt.xlabel('Date')
plt.ylabel('Production (MW)')
plt.show()


### Quelles sont les sources de production d'énergie les plus corrélées ?

In [ ]:
df_energies= df_selection[["Charbon","Gaz","Fioul","Nucléaire","Eolien","Solaire","Hydraulique","Bioénergies","Consommation"]]
print(df_energies["Nucléaire"].mean())
# df_energies["Nucléaire"]/= df_energies["Nucléaire"].mean()
print(df_energies["Nucléaire"].std())
# print(df_energies.corr())
# df_energies

### Quelles est la proportion d'énergie renouvelable produite ?

In [ ]:
df_energies= df_selection[["Charbon","Gaz","Fioul","Nucléaire","Eolien","Solaire","Hydraulique","Bioénergies"]]
total_energies = sum(df_energies.sum(axis=1))

name_col =["Charbon","Gaz","Fioul","Nucléaire","Eolien","Solaire","Hydraulique","Bioénergies"]
sum_dic = {}
pourcent_dic = {}
for col in name_col:
    sum_dic[col] = sum(df_energies[col])
    pourcent_dic[col] = sum_dic[col]/total_energies*100
print(pourcent_dic)
renouvelable = ["Eolien","Solaire","Hydraulique","Bioénergies"]
pourcent_renouvelable = sum([pourcent_dic[nrj12] for nrj12 in renouvelable])
print(f"{pourcent_renouvelable}%")


Les énergies renouvelables représente plus d'un quart de la production total d'énergie.

### Quelles sont les tendances de méthode de production d'énergie au cours de l'année ?
TODO: mathieu
separer données en plusieurs periode et les plot

# 4. Prédiction

In [ ]:
df.query('Charbon > 16').head()